Starter Code

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,polyarnyy,69.1989,33.4478,-0.93,72,0,9.58,RU,1682409403
1,1,linyi,35.0631,118.3428,15.76,59,90,5.24,CN,1682409129
2,2,floral city,28.7500,-82.2968,20.01,97,78,2.55,US,1682409404
3,3,west island,-12.1568,96.8225,28.99,74,75,8.75,CC,1682409404
4,4,santa luzia,-19.7697,-43.8514,18.50,77,0,2.06,BR,1682409404


Step 1: Create a map that displays a point for every city 

In [14]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 550,
    frame_height = 400,
    size = "Humidity",
    color = "City",
    alpha = 0.5,    
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

Step 2: Narrow down the DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"] >=21) & (city_data_df["Max Temp"] <=30)]
ideal_city_df = ideal_city_df.loc[(ideal_city_df["Wind Speed"] <=4.5)]
ideal_city_df = ideal_city_df.loc[(ideal_city_df["Cloudiness"] <=5)]

# Drop any rows with null values
ideal_city_df.dropna(how="any")

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,timimoun,29.2639,0.2310,27.46,14,4,3.47,DZ,1682409418
68,68,buka,40.8108,69.1986,27.27,24,2,2.47,UZ,1682409421
129,129,hassi messaoud,31.6804,6.0729,27.38,18,4,2.53,DZ,1682409443
150,150,puerto ayora,-0.7393,-90.3518,22.53,97,5,1.99,EC,1682409447
204,204,ankang,32.6800,109.0172,21.27,46,0,0.98,CN,1682409460
216,216,puerto baquerizo moreno,-0.9000,-89.6000,25.21,82,1,3.58,EC,1682409463
226,226,abu al matamir,30.9102,30.1744,21.95,30,0,2.01,EG,1682409464
253,253,tete,-16.1564,33.5867,26.05,69,4,0.00,MZ,1682409470
332,332,miandrivazo,-19.5167,45.4667,28.68,51,0,0.86,MG,1682409488
343,343,luderitz,-26.6481,15.1594,22.28,60,0,3.94,NaN,1682409490


Step 3: Create a new Dataframe 

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\rebec\AppData\Local\Temp\ipykernel_21928\1672974955.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
62,timimoun,DZ,29.2639,0.2310,14,
68,buka,UZ,40.8108,69.1986,24,
129,hassi messaoud,DZ,31.6804,6.0729,18,
150,puerto ayora,EC,-0.7393,-90.3518,97,
204,ankang,CN,32.6800,109.0172,46,
216,puerto baquerizo moreno,EC,-0.9000,-89.6000,82,
226,abu al matamir,EG,30.9102,30.1744,30,
253,tete,MZ,-16.1564,33.5867,69,
332,miandrivazo,MG,-19.5167,45.4667,51,
343,luderitz,NaN,-26.6481,15.1594,60,


Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates

In [17]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty & convert to JSON format
    name_address = requests.get(base_url, params=params).json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
timimoun - nearest hotel: أناف
buka - nearest hotel: No hotel found
hassi messaoud - nearest hotel: AMC
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
ankang - nearest hotel: 金友商务酒店
puerto baquerizo moreno - nearest hotel: Dory's House
abu al matamir - nearest hotel: No hotel found
tete - nearest hotel: Zambezi Hotel
miandrivazo - nearest hotel: Baobab
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
mersa matruh - nearest hotel: صقور


,City,Country,Lat,Lng,Humidity,Hotel Name
62,timimoun,DZ,29.2639,0.2310,14,أناف
68,buka,UZ,40.8108,69.1986,24,No hotel found
129,hassi messaoud,DZ,31.6804,6.0729,18,AMC
150,puerto ayora,EC,-0.7393,-90.3518,97,Hostal La Mirada De Solitario George
204,ankang,CN,32.6800,109.0172,46,金友商务酒店
216,puerto baquerizo moreno,EC,-0.9000,-89.6000,82,Dory's House
226,abu al matamir,EG,30.9102,30.1744,30,No hotel found
253,tete,MZ,-16.1564,33.5867,69,Zambezi Hotel
332,miandrivazo,MG,-19.5167,45.4667,51,Baobab
343,luderitz,NaN,-26.6481,15.1594,60,Seaview Hotel Zum Sperrgebiet


Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map

In [32]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 550,
    frame_height = 400,
    size = 50,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]    
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)